# AlphaZero Chess Training on Google Colab

Train an AlphaZero chess engine using a C++ MCTS backend with GPU-accelerated neural networks.

**Requirements:**
- Colab GPU runtime (Runtime → Change runtime type → T4 GPU)
- Google Drive for persistent checkpoint storage

**What this notebook does:**
1. Builds the C++ backend (MCTS + self-play engine) from source
2. Runs the AlphaZero training loop with parallel self-play
3. Saves checkpoints to Google Drive so they persist across sessions

## 1. GPU Check & Google Drive Mount

In [ ]:
# Verify GPU is available and mount Google Drive for checkpoint persistence
import torch

if not torch.cuda.is_available():
    raise RuntimeError(
        "No GPU detected! Go to Runtime > Change runtime type > T4 GPU"
    )

print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"CUDA: {torch.version.cuda}")
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader

# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

import os
DRIVE_DIR = "/content/drive/MyDrive/alphazero-chess"
os.makedirs(DRIVE_DIR, exist_ok=True)
print(f"\nDrive output directory: {DRIVE_DIR}")

## 2. Clone Repository

In [ ]:
import os

REPO_DIR = "/content/alpha-zero-chess"

if not os.path.exists(REPO_DIR):
    !git clone https://github.com/lirockyzhang/alpha-zero-chess.git {REPO_DIR}
else:
    print(f"Repository already cloned at {REPO_DIR}")
    !cd {REPO_DIR} && git pull

os.chdir(REPO_DIR)
print(f"Working directory: {os.getcwd()}")

## 3. Install Dependencies

In [ ]:
# torch and numpy are pre-installed on Colab
# We only need pybind11 (for building C++) and python-chess (for the training script)
!pip install pybind11 python-chess -q

# Verify imports
import pybind11, chess, torch, numpy
print(f"pybind11:     {pybind11.__version__}")
print(f"python-chess: {chess.__version__}")
print(f"torch:        {torch.__version__}")
print(f"numpy:        {numpy.__version__}")

## 4. Build C++ Module

Compiles the C++ MCTS engine and Python bindings. This takes ~2 minutes on Colab.

In [ ]:
import os, subprocess

BUILD_DIR = os.path.join(REPO_DIR, "alphazero-cpp", "build")
os.makedirs(BUILD_DIR, exist_ok=True)

# Get pybind11 cmake directory (overrides the hardcoded Windows path in CMakeLists.txt)
pybind11_dir = subprocess.check_output(
    ["python3", "-c", "import pybind11; print(pybind11.get_cmake_dir())"]
).decode().strip()
print(f"pybind11 cmake dir: {pybind11_dir}")

# Configure
!cd {BUILD_DIR} && cmake .. \
    -DCMAKE_BUILD_TYPE=Release \
    -Dpybind11_DIR="{pybind11_dir}"

# Build using all available cores
import multiprocessing
n_cores = multiprocessing.cpu_count()
print(f"\nBuilding with {n_cores} cores...")
!cd {BUILD_DIR} && cmake --build . --config Release -j{n_cores}

# On Linux, CMake puts the .so directly in build/ (not build/Release/)
# The training script expects it in build/Release/, so create that structure
RELEASE_DIR = os.path.join(BUILD_DIR, "Release")
os.makedirs(RELEASE_DIR, exist_ok=True)

import glob
so_files = glob.glob(os.path.join(BUILD_DIR, "alphazero_cpp*.so"))
if so_files:
    for so in so_files:
        target = os.path.join(RELEASE_DIR, os.path.basename(so))
        if not os.path.exists(target):
            os.symlink(so, target)
            print(f"Symlinked: {os.path.basename(so)} -> Release/")

# Verify the module loads
import sys
sys.path.insert(0, RELEASE_DIR)
sys.path.insert(0, BUILD_DIR)
import alphazero_cpp
print(f"\nalphazero_cpp loaded successfully!")

## 5. Configure Training Parameters

Adjust these parameters before starting training. The defaults are a good starting point for full training.

In [ ]:
# @title Training Configuration { run: "auto" }

# --- Network Architecture ---
FILTERS = 192          # @param {type: "integer"}
BLOCKS = 15            # @param {type: "integer"}

# --- Training Loop ---
ITERATIONS = 100       # @param {type: "integer"}
GAMES_PER_ITER = 50    # @param {type: "integer"}
SIMULATIONS = 800      # @param {type: "integer"}
EPOCHS = 5             # @param {type: "integer"}
LR = 0.001             # @param {type: "number"}
TRAIN_BATCH = 256      # @param {type: "integer"}
BUFFER_SIZE = 100000   # @param {type: "integer"}

# --- Parallel Self-Play ---
WORKERS = 16           # @param {type: "integer"}
EVAL_BATCH = 512       # @param {type: "integer"}

# --- Checkpointing ---
SAVE_INTERVAL = 5      # @param {type: "integer"}

# --- Resume from previous run (leave empty for fresh start) ---
RESUME_PATH = ""       # @param {type: "string"}

# --- Checkpoint directory setup ---
import os

SAVE_DIR = os.path.join(DRIVE_DIR, "checkpoints")
os.makedirs(SAVE_DIR, exist_ok=True)

# Create symlink so the training script's default path also works
LOCAL_CKPT = os.path.join(REPO_DIR, "checkpoints")
if os.path.islink(LOCAL_CKPT):
    os.remove(LOCAL_CKPT)
if not os.path.exists(LOCAL_CKPT):
    os.symlink(SAVE_DIR, LOCAL_CKPT)
    print(f"Symlinked checkpoints/ -> {SAVE_DIR}")
else:
    print(f"Note: {LOCAL_CKPT} already exists as a directory, using --save-dir instead")

print(f"\nCheckpoints will be saved to: {SAVE_DIR}")
print(f"Network: {FILTERS} filters, {BLOCKS} blocks")
print(f"Training: {ITERATIONS} iterations, {GAMES_PER_ITER} games/iter, {WORKERS} workers")
if RESUME_PATH:
    print(f"Resuming from: {RESUME_PATH}")

## 6. Run Training

This starts the AlphaZero training loop. Progress is printed every 30 seconds.

Checkpoints are saved to Google Drive every `SAVE_INTERVAL` iterations, so they persist even if Colab disconnects.

In [ ]:
import os

cmd = (
    f"python {os.path.join(REPO_DIR, 'alphazero-cpp', 'scripts', 'train.py')}"
    f" --iterations {ITERATIONS}"
    f" --games-per-iter {GAMES_PER_ITER}"
    f" --simulations {SIMULATIONS}"
    f" --workers {WORKERS}"
    f" --eval-batch {EVAL_BATCH}"
    f" --filters {FILTERS}"
    f" --blocks {BLOCKS}"
    f" --train-batch {TRAIN_BATCH}"
    f" --lr {LR}"
    f" --epochs {EPOCHS}"
    f" --buffer-size {BUFFER_SIZE}"
    f" --save-dir {SAVE_DIR}"
    f" --save-interval {SAVE_INTERVAL}"
    f" --progress-interval 30"
    f" --no-visualization"
    f" --no-eval"
    f" --device cuda"
)

if RESUME_PATH:
    cmd += f" --resume {RESUME_PATH}"

print("Training command:")
print(cmd)
print("\n" + "=" * 60)
print("Starting training...")
print("=" * 60 + "\n")

!{cmd}

## 7. Results & Next Steps

In [ ]:
import os, json

print("=" * 60)
print("Saved checkpoints on Google Drive:")
print("=" * 60)

if os.path.exists(SAVE_DIR):
    for run_dir in sorted(os.listdir(SAVE_DIR)):
        run_path = os.path.join(SAVE_DIR, run_dir)
        if os.path.isdir(run_path):
            files = os.listdir(run_path)
            pt_files = sorted([f for f in files if f.endswith(".pt")])
            print(f"\n  {run_dir}/")
            for f in pt_files:
                size_mb = os.path.getsize(os.path.join(run_path, f)) / (1024 * 1024)
                print(f"    {f}  ({size_mb:.1f} MB)")

            # Show training metrics if available
            metrics_path = os.path.join(run_path, "training_metrics.json")
            if os.path.exists(metrics_path):
                with open(metrics_path) as mf:
                    metrics = json.load(mf)
                n_iters = len(metrics)
                if n_iters > 0:
                    last = metrics[-1]
                    print(f"    Iterations completed: {n_iters}")
                    print(f"    Final loss: {last.get('total_loss', 'N/A')}")
else:
    print("  No checkpoints found.")

print("\n" + "=" * 60)
print("To resume training in a new session:")
print("=" * 60)
print(f"  1. Run cells 1-5 again")
print(f"  2. Set RESUME_PATH to the run directory, e.g.:")
print(f"     RESUME_PATH = \"{SAVE_DIR}/<run_directory>\"")
print(f"  3. Run cell 6 to continue training")